In [2]:
import warnings
warnings.filterwarnings('ignore')
import pickle
import re
import gc
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import tensorflow as tf
import tensorflow_hub as hub
from keras import layers
from keras.models import Model
from keras import backend as K
from itertools import chain

W0420 21:39:39.313423  8328 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14
Using TensorFlow backend.


In [34]:
#Reading train and test data
df = pd.read_csv("MyData.csv")

def read_label(raw_data):
    #logging.info("reading raw data...this may take a while")
    l = len(raw_data)
    label_list = [] #np.empty((raw_data.shape[0],len(tagCol)))
    ll =[]
    tagCol = [col for col in raw_data.columns if 'Tag_' in col]
    for j in range(0,raw_data.shape[0]):
        for i in range(0,len(tagCol)):
            if(raw_data.loc[raw_data.index[j],tagCol[i]]==1):
                ll.append(tagCol[i])
                #print(ll)
        label_list.append(ll)
        ll = []
        #print(label_list)
    return label_list

label_list = read_label(df)
#label_list

df['tagslist'] = label_list

train = df.loc[0:50,]
test = df.loc[51:len(df),]

#train_data=pd.read_csv("train.csv")
#test_data=pd.read_csv("test.csv")

In [39]:
tags=list(train['tagslist'])
tags.append(test['tagslist'])
#Saving pre-processed tags into "tags.pkl"
with open('tags.pkl', 'wb') as f:
    pickle.dump(tags, f)

paras=list(train['text'])
paras.append(test['text'])

paras=paras_text_processing(paras)

#Splitting paras on next line
for i in range(0,len(paras)):
    paras[i]=paras[i].split()

#Separating test data from paras
test_paras=paras[len(tags):]

#Saving pre-processed test data into file "test_paras.txt"
with open("test_paras.txt", "w") as f:
    for para in test_paras:
        f.write(str(para) +"\n")

#Separating train data from paras
paras=paras[0:len(tags)]

#Saving pre-processed train data into file "train_paras.txt"
with open("train_paras.txt", "w") as f:
    for para in paras:
        f.write(str(para) +"\n")


In [36]:
def paras_text_processing(paras):
    ''' Preprocessing on data - remove stop words, punctuations, single characters and numbers '''
    
    #NLTK stop words
    stop = set(stopwords.words('english'))
    
    #Removing stop words
    for i in range(0,len(paras)):
        paras[i]=[word for word in paras[i] if word not in stop]
        paras[i]=" ".join(word for word in paras[i])
        
    #Regex to remove punctuation, single characters and numbers
    for i in range(0,len(paras)):
        paras[i] = re.sub(r'\b\w{1,1}\b', '', re.sub('[^A-Za-z0-9]+', ' ', paras[i]))
        
    return paras       